### Extract DEA Water Observation (WO) statistics and attach to shapefile of DEA waterbodies for specific area
##### DEA waterbodies dataset can be downloaded from: https://cmi.ga.gov.au/data-products/dea/693/dea-waterbodies-landsat#acces
##### This notebook is designed to be run on DEA Sandbox https://app.sandbox.dea.ga.gov.au/

In [2]:
import datacube
from datacube.utils import masking
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, 'Tools/')

from datacube.utils.cog import write_cog
from dea_tools.datahandling import load_ard
from datetime import datetime
from pytz import timezone
import geopandas as gpd
import gc
import time
import numpy as np
import rioxarray as riox

In [3]:
dc = datacube.Datacube(app='GDEs')

In [4]:
#set infile to shapefile used as input (eg waterbodies/GDEs) to attach WO statistics to
infile = "WaterBodies.shp"
prefix = "WB"

#read shape file into dataframe
gdf = gpd.read_file(infile)

#set coordinate reference system to be coordinate system of shapefile
crs = gdf.crs

#change projection to albers (to assist with buffering in metres)
gdf = gdf.to_crs("epsg:3577")
crs = gdf.crs

#set bounds to be extent of shapefile
bounds = gdf.bounds

#to ensure tiff will be outside all polygons, add 30m to bounds (had issue with poly being 2cm outside tiff)
minx = bounds.minx.min() -120   
maxx = bounds.maxx.max() +120
miny = bounds.miny.min() -120  
maxy = bounds.maxy.max() +120


In [ ]:
#check coordinates are in Albers (m) and coordinate system is Albers (3577)
print (minx, miny, maxx, maxy)
print (crs)

In [ ]:
# Extract the Water Observation (WO) collection 3 data
ds = dc.load(product='ga_ls_wo_fq_myear_3',
                  x=(minx, maxx),
                  y=(maxy, miny),
                  crs = 'EPSG:3577',
                  output_crs='EPSG:3577',
                  resolution=(-30, 30)
            )
ds

In [ ]:
# save the raster as geotiff
write_cog(geo_im= ds.frequency, fname='data/{}_WO_fq_myear_c3.tif'.format(prefix), overwrite=True)


In [ ]:
# open WOfS raster and add to dictionary
wo_fq_myear_raster = riox.open_rasterio("data/{}_WO_fq_myear_c3.tif".format(prefix))

# create array of floats for writing the wofs filtered summary
arrays = {"wo_fq_myear_mean": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_median": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_min": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_max": np.nan * np.ones(shape = len(gdf), dtype = float),}           

In [ ]:
#check output filename is correct
print ("data/{}_WO_fq_myear_c3.tif".format(prefix))

In [ ]:
#initialise time so can calculate how long process takes
start_time = time.time()

#datetime.now() gets current time; astimezone is function to convert to timezone; timezone('Australia/Sydney') is the timezone to conver to
now = datetime.now().astimezone(timezone('Australia/Sydney'))  #setting time to be Sydney timezone
timef = now.strftime("%d-%m-%Y %H:%M:%S")  #formatting the time
print (timef)
# iterate through the geodataframe and get the geometry
for i, (index, row) in enumerate(gdf.iterrows()):
    if i%100 == 0:
        print(i)
    poly = row.geometry
    ######################################################################################################
    #CHECK ANY POLYS THAT GET PULLED OUT FOR BEING TOO SMALL AND DELETE FROM SHAPEFILE IF NOT NEEDED
    #> 800m seems to grab values that are more than half the pixel, so those < 800m should be ok to exclude
    #######################################################################################################
    if row.geometry.area > 800: 
        clipped_raster = wo_fq_myear_raster.rio.clip([poly])
    # extract array
        arr = clipped_raster.values[0]
        
        # Flag polygons with all No Data, probably due to shadows in Water Observations dataset
        if np.all(np.isnan(arr)):
            print("Polygon with ufi {} contains only NaN data, likely due to known shadows in Water Observations dataset".format(row["ufi"]))
    
    # remove null values and save the mean to the array
        arrays["wo_fq_myear_mean"][i] = np.nanmean(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_median"][i] = np.nanmedian(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_max"][i] = np.nanmax(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_min"][i] = np.nanmin(arr[arr != clipped_raster._FillValue])
    else:
        #print UFI of polygons in shapefile that are too small to keep track of small polys in case they are needed, 
        #in which case reduce geometry.area in if statement above be a value < 800
        print("Polygon too small to process \n","UFI",row.ufi) #
        continue
print("Time elapsed {} seconds".format(time.time() - start_time))


In [ ]:
# add the arrays to the geodataframe
gdf['wo_mean'] = arrays['wo_fq_myear_mean']
gdf['wo_p50'] = arrays['wo_fq_myear_median']
gdf['wo_min'] = arrays['wo_fq_myear_min']
gdf['wo_max'] = arrays['wo_fq_myear_max']
#gdf['tcw_p50'] = arrays['TCW_PC_50']


In [ ]:
# save the geodataframe to a file
#to automatically create filename with same name as input file but with 'stat's added, need to strip .shp from name
filename = (infile.rstrip(".shp"))
filename = (filename + '_WO_fq_myear_c3.shp')
###not sure if qbove will work as has 'r' below before path in quotes

gdf.to_file(filename, index =False)
#original code to write file (use if can't get above working)
#gdf.to_file(r"data/UD_Waterbodies_intersect_aquaticGDEs_TCW.shp", index =False)